# Matrix elements of field tensor operators

*Authors: Andrey Yachmenev & Cem Saribal*

Here you will learn how to obtain matrix representations of field tensor operators, convert between different forms, and apply filters to select desired subspaces of basis states.

For start, let's generate dipole moment and polarizability tensors for water molecule, for details, see "Rotational dynamics Quickstart"

In [42]:
from richmol.rot import Molecule, solve, LabTensor
import numpy as np

water = Molecule()

water.XYZ = ("bohr",
             "O",  0.00000000,   0.00000000,   0.12395915,
             "H",  0.00000000,  -1.43102686,  -0.98366080,
             "H",  0.00000000,   1.43102686,  -0.98366080)

# molecular-frame dipole moment (au)
water.dip = [0, 0, -0.7288]

# molecular-frame polarizability tensor (au)
water.pol = [[9.1369, 0, 0], [0, 9.8701, 0], [0, 0, 9.4486]]

Jmax = 2
water.sym = "C2v"

sol = solve(water, Jmax=Jmax)

# laboratory-frame dipole moment operator
dip = LabTensor(water.dip, sol)

# laboratory-frame polarizability tensor
pol = LabTensor(water.pol, sol)

# field-free Hamiltonian
h0 = LabTensor(water, sol)

## Getting matrix elements

You can get the full matrix representation of a tensor by calling `tomat` method, where you can specify the form of the output matrix (`form` keyword) as well as the desired Cartesian component of tensor (`cart` keyword). For example, 2D matrix representations of the $X$, $Y$, and $Z$ dipole moment components can be obtained as follows

In [43]:
mu_x = dip.tomat(form="full", cart="x")
mu_y = dip.tomat(form="full", cart="y")
mu_z = dip.tomat(form="full", cart="z")

print("matrix elements of mu_x operator")
print(mu_x)

matrix elements of mu_x operator
  (0, 1)	(-0-0.29753135409006326j)
  (0, 3)	0.29753135409006326j
  (1, 0)	0.29753135409006326j
  (1, 10)	-0.20052055607190472j
  (1, 11)	-0.2569462873688082j
  (1, 14)	0.08186217421921829j
  (1, 15)	0.10489788255935244j
  (2, 12)	-0.1417894449657412j
  (2, 13)	-0.18168846219919169j
  (2, 16)	0.1417894449657412j
  (2, 17)	0.18168846219919169j
  (3, 0)	-0.29753135409006326j
  (3, 14)	-0.08186217421921829j
  (3, 15)	-0.10489788255935244j
  (3, 18)	0.20052055607190472j
  (3, 19)	0.2569462873688082j
  (4, 8)	(-9.258728549113566e-50+0.25766971106437786j)
  (4, 30)	(4.370915472751295e-35+0.28226302627159655j)
  (4, 32)	(-1.784418769512764e-35-0.11523339793653577j)
  (5, 7)	(-9.258728549113566e-50+0.25766971106437786j)
  (5, 9)	(-9.258728549113566e-50+0.25766971106437786j)
  (5, 31)	(3.090703970775646e-35+0.1995900999548826j)
  (5, 33)	(-3.090703970775646e-35-0.1995900999548826j)
  (6, 8)	(-9.258728549113566e-50+0.25766971106437786j)
  (6, 32)	(1.78441876951276

For many tensor operators selection rules are such that only certain $J$ quanta and certain symmetries are coupled. For example, for the dipole moment operator it holds $|J' - J|\leq 1$, also in the $C_{2v}$ molecular symmetry group $A_1$ symmetry states are coupled only with $A_2$ and $B_1$ only with $B_2$.
It is therefore sometimes convenient to use block representation of tensor matrix elements, where only non-zero blocks, corresponding to different pairs of bra and ket $J$ quanta and symmetries, are stored. This representation can be obtained using `tomat` method with `form="block"` keyword (whcih is a default value)

In [47]:
mu_x = dip.tomat(form="block", cart="x")
mu_y = dip.tomat(form="block", cart="y")
mu_z = dip.tomat(form="block", cart="z")

print("matrix elements of mu_x operator")
for (J1, J2) in mu_x.keys():
    for (sym1, sym2) in mu_x[(J1, J2)].keys():
        mat = mu_x[(J1, J2)][(sym1, sym2)]
        # you may notice |J1-J2|<=1 and A1<->A2, B1<->B2 selection rules
        print("(J', J) =", (J1, J2), "(sym', sym) =", (sym1, sym2), "\n", mat)

matrix elements of mu_x operator
(J', J) = (0, 1) (sym', sym) = ('A1', 'A2') 
   (0, 0)	(-0-0.29753135409006326j)
  (0, 2)	0.29753135409006326j
(J', J) = (1, 2) (sym', sym) = ('A2', 'A1') 
   (0, 0)	-0.20052055607190472j
  (0, 1)	-0.2569462873688082j
  (0, 4)	0.08186217421921829j
  (0, 5)	0.10489788255935244j
  (1, 2)	-0.1417894449657412j
  (1, 3)	-0.18168846219919169j
  (1, 6)	0.1417894449657412j
  (1, 7)	0.18168846219919169j
  (2, 4)	-0.08186217421921829j
  (2, 5)	-0.10489788255935244j
  (2, 8)	0.20052055607190472j
  (2, 9)	0.2569462873688082j
(J', J) = (1, 2) (sym', sym) = ('B1', 'B2') 
   (0, 0)	(4.370915472751295e-35+0.28226302627159655j)
  (0, 2)	(-1.784418769512764e-35-0.11523339793653577j)
  (1, 1)	(3.090703970775646e-35+0.1995900999548826j)
  (1, 3)	(-3.090703970775646e-35-0.1995900999548826j)
  (2, 2)	(1.784418769512764e-35+0.11523339793653577j)
  (2, 4)	(-4.370915472751295e-35-0.28226302627159655j)
(J', J) = (1, 2) (sym', sym) = ('B2', 'B1') 
   (0, 0)	(-4.370915472751295e-3

By default, the output matrix (or matrix blocks in dict values when `form="block"`) is a `scipy.sparse.spmatrix` sparse object, which you can convert to an array by using the standard `toarray()` method. You can also directly get dense matrix representation from `tomat` by specifying `repres="dense"` keyword

In [48]:
# 2D matrix form
mu_x = dip.tomat(form="full", cart="x") # sparse matrix
mu_x2 = dip.tomat(form="full", repres="dense", cart="x") # dense matrix

# compare two results for 2D matrix form
print("2D, sparse == dense ?:", np.allclose(mu_x.toarray(), mu_x2))

# block form
mu_x = dip.tomat(form="block", cart="x") # blocks are sparse matrices
mu_x2 = dip.tomat(form="block", repres="dense", cart="x") # blocks are dense matrices

# compare two results for block matrix form
for (J1, J2) in mu_x.keys():
    for (sym1, sym2) in mu_x[(J1, J2)].keys():
        mat = mu_x[(J1, J2)][(sym1, sym2)].toarray()
        mat2 = mu_x2[(J1, J2)][(sym1, sym2)]
        print("block", (J1, J2, sym1, sym2), "sparse == dense ?:", np.allclose(mat, mat2))

2D, sparse == dense ?: True
block (0, 1, 'A1', 'A2') sparse == dense ?: True
block (1, 2, 'A2', 'A1') sparse == dense ?: True
block (1, 2, 'B1', 'B2') sparse == dense ?: True
block (1, 2, 'B2', 'B1') sparse == dense ?: True
block (2, 1, 'A1', 'A2') sparse == dense ?: True
block (2, 1, 'B2', 'B1') sparse == dense ?: True
block (2, 1, 'B1', 'B2') sparse == dense ?: True
block (1, 1, 'B1', 'B2') sparse == dense ?: True
block (1, 1, 'B2', 'B1') sparse == dense ?: True
block (2, 2, 'A2', 'A1') sparse == dense ?: True
block (2, 2, 'A1', 'A2') sparse == dense ?: True
block (2, 2, 'B2', 'B1') sparse == dense ?: True
block (2, 2, 'B1', 'B2') sparse == dense ?: True
block (1, 0, 'A2', 'A1') sparse == dense ?: True


You may choose any sparse representation of the `tomat` output matrix (or matrix blocks in dict values when `form="block"`) that is supported by `scipy.sparse` module

In [49]:
mu_x = dip.tomat(form="full", repres="csr_matrix", cart="x") # csr matrix
print("2D, format:", mu_x.getformat())

mu_x = dip.tomat(form="full", repres="csc_matrix", cart="x") # csc matrix
print("2D, format:", mu_x.getformat())

mu_x = dip.tomat(form="full", repres="coo_matrix", cart="x") # coo matrix
print("2D, format:", mu_x.getformat())

mu_x = dip.tomat(form="block", repres="lil_matrix", cart="x") # blocks are lil matrices
for (J1, J2) in mu_x.keys():
    for (sym1, sym2) in mu_x[(J1, J2)].keys():
        print("block", (J1, J2, sym1, sym2), "format:", mu_x[(J1, J2)][(sym1, sym2)].getformat())

2D, format: csr
2D, format: csc
2D, format: coo
block (0, 1, 'A1', 'A2') format: lil
block (1, 2, 'A2', 'A1') format: lil
block (1, 2, 'B1', 'B2') format: lil
block (1, 2, 'B2', 'B1') format: lil
block (2, 1, 'A1', 'A2') format: lil
block (2, 1, 'B2', 'B1') format: lil
block (2, 1, 'B1', 'B2') format: lil
block (1, 1, 'B1', 'B2') format: lil
block (1, 1, 'B2', 'B1') format: lil
block (2, 2, 'A2', 'A1') format: lil
block (2, 2, 'A1', 'A2') format: lil
block (2, 2, 'B2', 'B1') format: lil
block (2, 2, 'B1', 'B2') format: lil
block (1, 0, 'A2', 'A1') format: lil


You can also convert between 2D and block matrix forms using `block_form` and `full_form` methods

In [50]:
# generate 2D matrix from `dip`
mu_x_2d = dip.tomat(form="full", repres="csr_matrix", cart="x")

# convert `mu_x_2d` matrix to block form
mu_x_block1 = dip.block_form(mu_x_2d)

# generate block matrix from `dip`
mu_x_block2 = dip.tomat(form="block", repres="csr_matrix", cart="x")

# compare two block matrices
for (J1, J2) in mu_x_block.keys():
    for (sym1, sym2) in mu_x_block[(J1, J2)].keys():
        mat = mu_x_block[(J1, J2)][(sym1, sym2)]
        mat2 = mu_x_block2[(J1, J2)][(sym1, sym2)]
        print("block", (J1, J2, sym1, sym2), "mu_x_block == mu_x_block2 ?:", np.allclose(mat.toarray(), mat2.toarray()))

block (0, 1, 'A1', 'A2') mu_x_block == mu_x_block2 ?: True
block (1, 2, 'A2', 'A1') mu_x_block == mu_x_block2 ?: True
block (1, 2, 'B1', 'B2') mu_x_block == mu_x_block2 ?: True
block (1, 2, 'B2', 'B1') mu_x_block == mu_x_block2 ?: True
block (2, 1, 'A1', 'A2') mu_x_block == mu_x_block2 ?: True
block (2, 1, 'B2', 'B1') mu_x_block == mu_x_block2 ?: True
block (2, 1, 'B1', 'B2') mu_x_block == mu_x_block2 ?: True
block (1, 1, 'B1', 'B2') mu_x_block == mu_x_block2 ?: True
block (1, 1, 'B2', 'B1') mu_x_block == mu_x_block2 ?: True
block (2, 2, 'A2', 'A1') mu_x_block == mu_x_block2 ?: True
block (2, 2, 'A1', 'A2') mu_x_block == mu_x_block2 ?: True
block (2, 2, 'B2', 'B1') mu_x_block == mu_x_block2 ?: True
block (2, 2, 'B1', 'B2') mu_x_block == mu_x_block2 ?: True
block (1, 0, 'A2', 'A1') mu_x_block == mu_x_block2 ?: True


In [51]:
# generate block matrix from `dip`
mu_x_block = dip.tomat(form="block", repres="csr_matrix", cart="x")

# convert `mu_x_block` to 2D form
mu_x_2d = dip.full_form(mu_x_block)

# generate 2D matrix from `dip`
mu_x_2d_2 = dip.tomat(form="full", repres="csr_matrix", cart="x")

print("mu_x_2d == mu_x_2d_2 ?:", np.allclose(mu_x_2d.toarray(), mu_x_2d_2.toarray()))

mu_x_2d == mu_x_2d_2 ?: True


## Getting assignments

In [55]:
#TODO

## Selecting subspaces of states

In [54]:
#TODO